In [15]:
import os
import json
import pandas as pd
import traceback

In [16]:
from langchain.chat_models import ChatOpenAI

In [ ]:
## %pip install python-dotenv 若有需要

In [17]:
from dotenv import load_dotenv 

load_dotenv()  # take environment variables from .env.

True

In [18]:
KEY=os.getenv("OPENAI_API_KEY")

In [20]:
%pip install -U langchain-openai --quiet

Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain_openai import ChatOpenAI

In [14]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

<font color="red">Deprecated 版本</font>
```python
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)
```

In [21]:
from langchain.llms import OpenAI ## Deprecated
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [42]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [24]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [25]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [26]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

e:\gitstore\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [27]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [28]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [29]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [30]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], 
                                        input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [31]:
file_path=r"E:\gitstore\mcqgen\data.txt"

In [36]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [37]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed

In [35]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON) ## 展示json.dumps的功能

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [44]:
NUMBER=5                     ## 問題數目
SUBJECT="machine learning"   ## 主題領域
TONE="simple"                ## 問題的難易度

In [45]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

e:\gitstore\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [46]:
print(f"Total Tokens:{cb.total_tokens}") ## input + output
print(f"Prompt Tokens:{cb.prompt_tokens}") ## input 
print(f"Completion Tokens:{cb.completion_tokens}") ## output
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1793
Prompt Tokens:1377
Completion Tokens:416
Total Cost:0.0028975


In [47]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [48]:
quiz=response.get("quiz") ## 等同 quiz=response["quiz"],此時 quiz 是 str 類型

In [52]:
quiz=json.loads(quiz) ## 把 str 類型的 quiz ，轉成 dict 類型的 quiz 

In [53]:
quiz_table_data = []
for key, value in quiz.items(): ## quiz是dict不是str類型
    mcq = value["mcq"]          ## 問題  
    options = " | ".join(       ## 選項
            [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]  ## 答案
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [55]:
quiz_table_data ## list of dict

[{'MCQ': 'Who coined the term machine learning in 1959?',
  'Choices': 'a: Donald Hebb | b: Walter Pitts | c: Arthur Samuel | d: Warren McCulloch',
  'Correct': 'c'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a: Cybernet | b: Cybertron | c: NeuralNet | d: DataLearn',
  'Correct': 'b'},
 {'MCQ': 'According to Tom M. Mitchell, what is the definition of machine learning algorithms?',
  'Choices': 'a: Improving with experience E in tasks T and performance measure P | b: Classifying data based on models | c: Making predictions for future outcomes | d: Recognizing patterns using reinforcement learning',
  'Correct': 'a'},
 {'MCQ': "What was the focus of Nilsson's book on Learning Machines during the 1960s?",
  'Choices': 'a: Machine learning for stock trading | b: Machine learning for pattern classification | c: Machine learning for computer vision | d: Machine learning for speech recognition',
  'Corr

In [56]:
quiz=pd.DataFrame(quiz_table_data)

In [57]:
quiz

,MCQ,Choices,Correct
0,Who coined the term machine learning in 1959?,a: Donald Hebb | b: Walter Pitts | c: Arthur S...,c
1,What was the name of the experimental 'learnin...,a: Cybernet | b: Cybertron | c: NeuralNet | d:...,b
2,"According to Tom M. Mitchell, what is the defi...",a: Improving with experience E in tasks T and ...,a
3,What was the focus of Nilsson's book on Learni...,a: Machine learning for stock trading | b: Mac...,b
4,What is one of the objectives of modern-day ma...,a: To develop new computer programs | b: To ma...,b


In [58]:
quiz.to_csv("machinelearning.csv",index=False)

In [59]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_18_2024_06_38_00'